## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import Image
import shutil
import copy
from numpy import int32
import os
from numpy import float32
import cv2
import re
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading Data

In [3]:
with open('/content/drive/MyDrive/MCM Practicum/Data/Train/annotations.json') as json_file:
    train_full_annotations_list=json.load(json_file)
print(len(train_full_annotations_list))

train_list_of_images=os.listdir("/content/drive/MyDrive/MCM Practicum/Data/Train/png/")
len(train_list_of_images)

train_annotations_list=[]
for i in range (len(train_full_annotations_list)):
  temp=(str(train_full_annotations_list[i]['image_index'])+'.png')
  if temp in train_list_of_images:
    train_annotations_list.append(train_full_annotations_list[i])
len(train_annotations_list)

157070


32658

In [4]:
#Splitting training dataset based on chart type
train_annotations_list_dot=[]
train_annotations_list_hbar=[]
train_annotations_list_line=[]
train_annotations_list_vbar=[]
i=0
for i in range(len(train_annotations_list)):
    test = train_annotations_list[i]
    if test['type']=='dot_line':
        train_annotations_list_dot.append(test)
    elif test['type']=='hbar_categorical':
        train_annotations_list_hbar.append(test)
    elif test['type']=='line':
        train_annotations_list_line.append(test)
    elif test['type']=='vbar_categorical':
        train_annotations_list_vbar.append(test)

print("Number of Dot Plots train:",len(train_annotations_list_dot))
print("Number of Line Plots train:",len(train_annotations_list_line))
print("Number of Horizontal Bar Plots train:",len(train_annotations_list_hbar))
print("Number of Vertical Bar Plots train:",len(train_annotations_list_vbar))


Number of Dot Plots train: 5421
Number of Line Plots train: 5377
Number of Horizontal Bar Plots train: 10948
Number of Vertical Bar Plots train: 10912


In [5]:
#removing annotation entries with negative coordinate values
train_subset=copy.copy(train_annotations_list)
train_delete_list=[]
for i in range(len(train_subset)):
    temp1=train_subset[i]['general_figure_info']['title']['bbox']
    temp4=train_subset[i]['general_figure_info']['x_axis']['label']['bbox']
    temp5=train_subset[i]['general_figure_info']['y_axis']['label']['bbox']
    if (((temp1['x'])<0) or ((temp1['y'])<0) or ((temp1['w'])<0) or ((temp1['h'])<0)) or (((temp4['x'])<0) or ((temp4['y'])<0) or ((temp4['w'])<0) or ((temp4['h'])<0)) or (((temp5['x'])<0) or ((temp5['y'])<0) or ((temp5['w'])<0) or ((temp5['h'])<0)):
        if (i) not in train_delete_list:
            train_delete_list.append(i)
    try:
        temp2=train_subset[i]['general_figure_info']['legend']['bbox']
    except KeyError:
        temp2=0
    if (temp2!=0):
        for k in range(len(train_subset[i]['general_figure_info']['legend']['items'])):
            temp3=train_subset[i]['general_figure_info']['legend']['items'][k]['label']['bbox']
            if (((temp2['x'])<0) or ((temp2['y'])<0) or ((temp2['w'])<0) or ((temp2['h'])<0)) or (((temp3['x'])<0) or ((temp3['y'])<0) or ((temp3['w'])<0) or ((temp3['h'])<0)):
                if (i) not in train_delete_list:
                    train_delete_list.append(i)
print(len(train_delete_list))


3232


In [19]:
train_subset_annotations_list=[]
dupicate=[]
negative=[]

for i in range (0,675):
    temp=train_annotations_list_dot[random.randint(0,len(train_annotations_list_dot)-1)]
    if temp in train_subset_annotations_list:
        dupicate.append(temp)
    elif i in train_delete_list:
        negative.append(temp)
    else:
        train_subset_annotations_list.append(temp)

for i in range (0,675):
    temp=train_annotations_list_line[random.randint(0,len(train_annotations_list_line)-1)]
    if temp in train_subset_annotations_list:
        dupicate.append(temp)
    elif i in train_delete_list:
        negative.append(temp)
    else:
        train_subset_annotations_list.append(temp)

for i in range (0,675):
    temp=train_annotations_list_hbar[random.randint(0,len(train_annotations_list_hbar)-1)]
    if temp in train_subset_annotations_list:
        dupicate.append(temp)
    elif i in train_delete_list:
        negative.append(temp)
    else:
        train_subset_annotations_list.append(temp)

for i in range (0,675):
    temp=copy.copy(train_annotations_list_vbar[random.randint(0,len(train_annotations_list_vbar)-1)])
    if temp in train_subset_annotations_list:
        dupicate.append(temp)
    elif i in train_delete_list:
        negative.append(temp)
    else:
        train_subset_annotations_list.append(temp)

print(len(train_subset_annotations_list))

2533


## Extracting Training Annotations

In [20]:
def convert_labels (path, test_bbox):
    test_bbox['w']=test_bbox['x']+test_bbox['w']
    test_bbox['h']=test_bbox['y']+test_bbox['h']
    #print(test_bbox)
    x1=test_bbox['x']
    y1=test_bbox['y']
    x2=test_bbox['w']
    y2=test_bbox['h']
    def sorting(l1, l2):
        if 11> 12:
            lmax, lmin = l1, l2
            return lmax, lmin
        else:
            lmax, lmin = l2, l1
            return lmax, lmin
    img=cv2.imread(path)
    size=img.shape
    #print(size)
    xmax, xmin = sorting(x1, x2)
    ymax, ymin = sorting (y1, y2)
    dw = 1./size[1]
    dh = 1./size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax-xmin
    h = ymax-ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [21]:
#YOLO_classes
#0=chart_title
#1=x_axis_label
#2=y_axis_label
#3=legend_elements

for f in os.listdir('/content/drive/MyDrive/text_extraction_dataset/images/train/'):
    os.remove(os.path.join('/content/drive/MyDrive/text_extraction_dataset/images/train/', f))
for f in os.listdir('/content/drive/MyDrive/text_extraction_dataset/labels/train/'):
    os.remove(os.path.join('/content/drive/MyDrive/text_extraction_dataset/labels/train/', f))

for k in range(len(train_subset_annotations_list)):
    path='/content/drive/MyDrive/MCM Practicum/Data/Train/png/'+str(train_subset_annotations_list[k]['image_index'])+'.png'
    m=0
    annotations_array=np.zeros((0,5), dtype=float)
    legend_bbox_list=[]
    try:
        legend_bbox=copy.copy(train_subset_annotations_list[k]['general_figure_info']['legend'])
        for i in range(len(train_subset_annotations_list[k]['general_figure_info']['legend']['items'])):
            l=len(train_subset_annotations_list[k]['general_figure_info']['legend']['items'])
            annotations_array = np.zeros((l,5), dtype=float)
        for i in range(len(legend_bbox['items'])):
            test_bbox=copy.copy(legend_bbox['items'][i]['label']['bbox'])
            annotations_array[m][0]=3
            test_bbox_yolo=convert_labels(path,test_bbox)
            annotations_array[m][1]=test_bbox_yolo[0]
            annotations_array[m][2]=test_bbox_yolo[1]
            annotations_array[m][3]=test_bbox_yolo[2]
            annotations_array[m][4]=test_bbox_yolo[3]
            m=m+1
    except KeyError:
        legend_bbox=0

    title_bbox_list=[]
    xaxis_bbox_list=[]
    yaxis_bbox_list=[]

    #inserting bboxes of title
    title_bbox=copy.copy(train_subset_annotations_list[k]['general_figure_info']['title']['bbox'])
    title_bbox=convert_labels(path,title_bbox)
    title_bbox_list.append(0)
    for i in range(len(title_bbox)):
        title_bbox_list.append(title_bbox[i])
    annotations_array=np.r_[annotations_array,[title_bbox_list]]

    #inserting bboxes of xaxis
    xaxis_bbox=copy.copy(train_subset_annotations_list[k]['general_figure_info']['x_axis']['label']['bbox'])
    xaxis_bbox=convert_labels(path,xaxis_bbox)
    xaxis_bbox_list.append(1)
    for i in range(len(xaxis_bbox)):
        xaxis_bbox_list.append(xaxis_bbox[i])
    annotations_array=np.r_[annotations_array,[xaxis_bbox_list]]

    #inserting bboxes of yaxis
    yaxis_bbox=copy.copy(train_subset_annotations_list[k]['general_figure_info']['y_axis']['label']['bbox'])
    yaxis_bbox=convert_labels(path,yaxis_bbox)
    yaxis_bbox_list.append(2)
    for i in range(len(yaxis_bbox)):
        yaxis_bbox_list.append(yaxis_bbox[i])
    annotations_array=np.r_[annotations_array,[yaxis_bbox_list]]

    name=('/content/drive/MyDrive/text_extraction_dataset/labels/train/'+str(train_subset_annotations_list[k]['image_index'])+'.txt')
    np.savetxt(name, annotations_array, delimiter=" ", fmt='%1.9f')
    source='/content/drive/MyDrive/MCM Practicum/Data/Train/png/'+str(train_subset_annotations_list[k]['image_index'])+'.png'
    dest=('/content/drive/MyDrive/text_extraction_dataset/images/train/'+str(train_subset_annotations_list[k]['image_index'])+'.png')
    shutil.copy2(source,dest)
